# Mini Project 1

In [1]:
print('Hello MP1!')

Hello MP1!


## 1. Dataset Preparation & Analysis (5pts)

1.2. Load the dataset. You can use `gzip.open` and `json.load` to do that.

In [2]:
import gzip
import json

dataset = gzip.open('goemotions.json.gz')
dataset_json = json.load(dataset)

# Close the gz dataset once your finished loading the data as a json object
dataset.close()


1.3. (5pts) Extract the posts and the 2 sets of labels (emotion and sentiment), then plot the distribution
of the posts in each category and save the graphic (a histogram or pie chart) in pdf. Do this for both
the emotion and the sentiment categories. You can use `matplotlib.pyplot` and `savefig` to do this.
This pre-analysis of the dataset will allow you to determine if the classes are balanced, and which
metric is more appropriate to use to evaluate the performance of your classifiers.

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter


numpy_dataset = np.array(dataset_json)

# Get column only for emotion and sentiment
emotion_dataset_col = numpy_dataset[:, 1]
sentiment_dataset_col = numpy_dataset[:, 2]

# Count the number of times each value appears
emotion_count = Counter(emotion_dataset_col)
sentiment_count = Counter(sentiment_dataset_col)
print(emotion_count)

# Save the data values as a histogram
print(emotion_count.values())

plt.hist(emotion_count.values())
plt.savefig('emotions_graph')

plt.close()


plt.hist(sentiment_count.values())
plt.savefig('sentiment_graph')

plt.close()


Counter({'neutral': 55298, 'approval': 11259, 'admiration': 10531, 'annoyance': 8342, 'disapproval': 7686, 'gratitude': 7075, 'amusement': 6130, 'curiosity': 5885, 'anger': 5202, 'love': 4957, 'confusion': 4938, 'realization': 4714, 'disappointment': 4706, 'optimism': 4519, 'joy': 4329, 'sadness': 3827, 'caring': 3523, 'surprise': 3472, 'excitement': 3020, 'disgust': 2914, 'desire': 2147, 'fear': 1778, 'remorse': 1510, 'embarrassment': 1433, 'nervousness': 796, 'relief': 788, 'pride': 690, 'grief': 351})
dict_values([3827, 55298, 4957, 7075, 7686, 6130, 4706, 4714, 10531, 8342, 4938, 4519, 3020, 3523, 1510, 4329, 11259, 1433, 3472, 5885, 5202, 351, 2914, 690, 2147, 788, 1778, 796])


## 2. Words as Features (35pts)

2.1. □ (5pts) Process the dataset using `feature extraction.text.CountVectorizer` to extract tokens/words
and their frequencies. Display the number of tokens (the size of the vocabulary) in the dataset.